### performance check

In [1]:
import os
import sys
sys.path.append('../')
import argparse
import asyncio
import json
import torch
import pandas as pd
import numpy as np
import random
from models.model import CossimRecommender
import pickle
from tqdm import tqdm


In [2]:
data_dir = '/opt/ml/final-project-level3-recsys-02/data/'
recommender = CossimRecommender(data_dir,'metapath_embeddings')

In [3]:
place = recommender.map_loader.place
id2place = recommender.id2place
place_id2idx = recommender.place2id
all_df = pd.read_csv(data_dir + 'all_df.csv')

In [4]:
def jaccard_similarity(list1, list2): 
    s1 = set(list1)
    s2 = set(list2)
    if len(s1.union(s2)):
        return float(len(s1.intersection(s2)) / len(s1.union(s2)))
    return 0

In [5]:
def get_feat_list(df, place_id, feature):
    return df[df.placeID == place_id][feature].tolist()

In [6]:
def compare_random(place_id, df, column):
    lonlan = place[place.placeID == id2place[place_id]]['map'].values[0]
    nearest_list = recommender.map_loader.filtermap(lonlan)

    
    m2v_topk = recommender.recommend(lonlan, place_id)
    random_topk = np.array(random.sample(nearest_list, k = min(len(nearest_list), 5)))

    m2v_score, random_score = 0, 0

    target_feat_list = get_feat_list(df, id2place[place_id], column)

    for placename in m2v_topk:
        rec_feat_list = get_feat_list(df, placename, column)
        m2v_score += jaccard_similarity(target_feat_list, rec_feat_list)

    for randomname in random_topk:
        rec_feat_list = get_feat_list(df, randomname, column)
        random_score += jaccard_similarity(target_feat_list, rec_feat_list)

    return m2v_score, random_score


In [7]:
def mean_score():
    mean_m2v, mean_random = 0, 0

    for i in tqdm(id2place.keys()):
        m2v_score, random_score = compare_random(i, all_df, 'feature')
        mean_m2v += m2v_score
        mean_random += random_score

    return mean_m2v, mean_random

In [8]:
mean_score()

  1%|          | 70/12677 [00:18<47:06,  4.46it/s]  